# Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Importing and Splitting data

In [17]:
file = pd.read_csv("audi.csv", quotechar='"', skipinitialspace=True)

modelEncoder = LabelEncoder()
modelEncoder.fit(file["model"])
file["model"] = modelEncoder.transform(file["model"])

transmissionEncoder = LabelEncoder()
transmissionEncoder.fit(file["transmission"])
file["transmission"] = transmissionEncoder.transform(file["transmission"])

fuelTypeEncoder = LabelEncoder()
fuelTypeEncoder.fit(file["fuelType"])
file["fuelType"] = fuelTypeEncoder.transform(file["fuelType"])

file = file.head(1000)
# X = file.drop(columns = ['price'])
# Y = file.price
# print(file)

X = file.drop(['price'], axis = 1).to_numpy()
Y = file['price'].values.reshape(-1,1)
# X = file[['year', 'mileage', 'tax', 'mpg', 'engineSize']].to_numpy() #dtype = 'int'

print(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 601)

[[0.000e+00 2.017e+03 1.000e+00 ... 1.500e+02 5.540e+01 1.400e+00]
 [5.000e+00 2.016e+03 0.000e+00 ... 2.000e+01 6.420e+01 2.000e+00]
 [0.000e+00 2.016e+03 1.000e+00 ... 3.000e+01 5.540e+01 1.400e+00]
 ...
 [5.000e+00 2.019e+03 2.000e+00 ... 1.500e+02 3.770e+01 3.000e+00]
 [1.000e+01 2.017e+03 2.000e+00 ... 1.500e+02 3.400e+01 3.000e+00]
 [4.000e+00 2.018e+03 2.000e+00 ... 1.500e+02 3.530e+01 3.000e+00]]


# Using Scikit-Learn

In [18]:
myTree = DecisionTreeRegressor(random_state = 601)
myTree.fit(X_train, Y_train)
print("R Squared: ", myTree.score(X_test, Y_test))

R Squared:  0.8469942517214841


# Visualising Decsion Tree:

In [19]:
treeView = DecisionTreeRegressor(random_state = 601, max_depth = 3)
treeView.fit(X_train,Y_train)

from sklearn.tree import plot_tree
plt.figure(figsize=(8,3), dpi = 150)
plot_tree(treeView, feature_names = X.columns, filled=True);

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

<Figure size 1200x450 with 0 Axes>

# Entropy

In [6]:
# def calc_total_entropy(train_data, label, class_list):
#     total_row = train_data.shape[0]
#     total_entr = 0 #l
    
#     for c in class_list: 
#         total_class_count = train_data[train_data[label] == c].shape[0] 
#         total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) 
#         total_entr += total_class_entr 
    
#     return total_entr



#  def entropy(self, y):
#         entropy = 0
#         labels = np.unique(y)
#         for label in labels:
#             label_examples = y[y == label]
#             pl = len(label_examples) / len(y)
#             entropy += -pl * np.log2(pl)
#         return entropy

# RMSE

In [7]:
def rmse(h, y):
  return sqrt(mean_squared_error(h, y))

In [77]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
#         self.var_red = var_red
        self.value = value #leaf

In [88]:
class DTRegressor():
    def __init__(self, minSamples = 2, maxDepth = 2):
        self.root = None
        self.minSamples = minSamples
        self.maxDepth = maxDepth
        
    def entropy(self, y):
        entropy = 0
        
        labels = np.unique(y)
        # loop over each unique value
        for label in labels:
            # find elements in y that are equal to that label
            label_examples = y[y == label]
            # get ratio of the label
            pl = len(label_examples) / len(y)
            # calculate entropy
            entropy += -pl * np.log2(pl)
        return entropy
        
    def infoGain(self, parent, leftNode, rightNode): #need to checkkk
        print("HERE 5")
        leftWeight = len(leftNode) / len(parent)
        rightWeight = len(rightNode) / len(parent)
        
#         reduction = np.var(parent) - (leftWeight * np.var(leftNode) + rightWeight * np.var(rightNode))
#         print ("REDUCTION: ", reduction)

        parent_entropy = self.entropy(parent)
        entropy_left, entropy_right = self.entropy(leftNode), self.entropy(rightNode)
        # calculate weighted entropy 
        weighted_entropy = leftWeight * entropy_left + rightWeight * entropy_right
        # calculate information gain 
        information_gain = parent_entropy - weighted_entropy
#         print(information_gain)

        return information_gain
#         return reduction
        
    def splitTree(self, dataset, feature, threshold):
        print("HERE 4")
        leftBranch = []
        rightBranch = []
        for i in dataset:
            if i[feature] <= threshold:
                leftBranch.append(i)
            else:
                rightBranch.append(i)
        rightBranch = np.array(rightBranch)
        leftBranch = np.array(leftBranch)
        return leftBranch, rightBranch
        
    def bestSplit(self, dataset, X):
        print("HERE 3")
        bestSplitt = {'gain':- 1}
        
        for i in range(X.shape[1]): #feature_index = i
            feature = dataset[:, i] #current feature selected
            thresholds = np.unique(feature)
#             print(thresholds)
            for j in thresholds: #threshold = j
                # get current split
                leftSide, rightSide = self.splitTree(dataset, i, j) #splits tree into 2 branches
            
                if (len(leftSide) > 0 and len(rightSide) > 0 ):
                    parent = dataset[:, -1]
                    leftNode = leftSide[:, -1]
                    rightNode = rightSide[:, -1]

                    gain = self.infoGain(parent, leftNode, rightNode) 
                    
                    if gain > bestSplitt["gain"]:
                        bestSplitt["feature"] = feature
                        bestSplitt["threshold"] = thresholds
                        bestSplitt["leftSide"] = leftSide
                        bestSplitt["rightSide"] = rightSide
                        bestSplitt["gain"] = gain
                
        return bestSplitt
        
    def treeBuild(self, dataset, currentDepth = 0):
        print("HERE 2")
        X = dataset[:,:-1]
        Y = dataset[:,-1] #Split training into features and labels
#         samplesNumb = X.shape[0]
#         featuresNumb = X.shape[1]
        
        #iterates until this condition is met
        if X.shape[0] >= self.minSamples and currentDepth <= self.maxDepth: 
            bestSplit = self.bestSplit(dataset, X)
            
#         print(n_features)
        
        
    def fit(self, X, Y):
        print("HERE 1")
        dataset = np.concatenate((X, Y), axis=1) #Joins training data back together
        self.root = self.treeBuild(dataset)

In [86]:
myTree = DTRegressor(2, 2) #create model instance
myTree.fit(X_train, Y_train)

HERE 1
HERE 2
HERE 3
HERE 4
HERE 5
0.4318866534327954
HERE 4
HERE 5
0.5822138621898212
HERE 4
HERE 5
0.6059304265860197
HERE 4
HERE 5
0.5766430844487811
HERE 4
HERE 5
0.5527702085970034
HERE 4
HERE 5
0.5364025997941129
HERE 4
HERE 5
0.5238743708984952
HERE 4
HERE 5
0.504861623453496
HERE 4
HERE 5
0.41302485754649076
HERE 4
HERE 5
0.23124291050053714
HERE 4
HERE 5
0.14676343258804447
HERE 4
HERE 5
0.12987400622947476
HERE 4
HERE 5
0.1111993902810795
HERE 4
HERE 5
0.1014822969423399
HERE 4
HERE 5
0.09429883273771633
HERE 4
HERE 5
0.07469078105307236
HERE 4
HERE 5
0.06699746895697523
HERE 4
HERE 4
HERE 5
0.10910057333985002
HERE 4
HERE 5
0.2210586422151648
HERE 4
HERE 5
0.3498323436642936
HERE 4
HERE 5
0.616398391246439
HERE 4
HERE 5
0.7470007733212078
HERE 4
HERE 5
0.6747312995026862
HERE 4
HERE 5
0.20277667716217618
HERE 4
HERE 4
HERE 5
0.4995946020332127
HERE 4
HERE 5
0.6168261726608097
HERE 4
HERE 4
HERE 5
0.17843286501467226
HERE 4
HERE 5
0.18457990572889837
HERE 4
HERE 5
0.190660143

HERE 5
0.7171298583694918
HERE 4
HERE 5
0.7155532814089813
HERE 4
HERE 5
0.7166322748078509
HERE 4
HERE 5
0.7177001893678181
HERE 4
HERE 5
0.7187570423328626
HERE 4
HERE 5
0.7161296673879161
HERE 4
HERE 5
0.7208376313419604
HERE 4
HERE 5
0.7202012507705131
HERE 4
HERE 5
0.7212140253677646
HERE 4
HERE 5
0.7196823085783599
HERE 4
HERE 5
0.7233398084161475
HERE 4
HERE 5
0.7233131778936706
HERE 4
HERE 5
0.7242821325278754
HERE 4
HERE 5
0.7215669876151454
HERE 4
HERE 5
0.7162608503413193
HERE 4
HERE 5
0.7145304347544776
HERE 4
HERE 5
0.7117826309548274
HERE 4
HERE 5
0.7100304867625624
HERE 4
HERE 5
0.7040044963572427
HERE 4
HERE 5
0.703237197349285
HERE 4
HERE 5
0.7049925962260222
HERE 4
HERE 5
0.7058638312552041
HERE 4
HERE 5
0.7050641286368879
HERE 4
HERE 5
0.7069203183281134
HERE 4
HERE 5
0.704086046756359
HERE 4
HERE 5
0.7037745395419055
HERE 4
HERE 5
0.6995251620177001
HERE 4
HERE 5
0.6963677978433633
HERE 4
HERE 5
0.6998304584246764
HERE 4
HERE 5
0.6979490870606115
HERE 4
HERE 5
0.700

HERE 5
0.521190803342205
HERE 4
HERE 5
0.5207376130710903
HERE 4
HERE 5
0.5190027833145763
HERE 4
HERE 5
0.5172554457644383
HERE 4
HERE 5
0.5148419324046785
HERE 4
HERE 5
0.5147296247870381
HERE 4
HERE 5
0.5129445540715896
HERE 4
HERE 5
0.5128200764687332
HERE 4
HERE 5
0.511009671300056
HERE 4
HERE 5
0.5048597272524216
HERE 4
HERE 5
0.5030238404098295
HERE 4
HERE 5
0.5011751564192135
HERE 4
HERE 5
0.4997996744415847
HERE 4
HERE 5
0.49519905451146506
HERE 4
HERE 5
0.49329879590922054
HERE 4
HERE 5
0.49138554481305974
HERE 4
HERE 5
0.48945926072960155
HERE 4
HERE 5
0.4875199026753245
HERE 4
HERE 5
0.48823409583618016
HERE 4
HERE 5
0.48893513156095736
HERE 4
HERE 5
0.4876099340147002
HERE 4
HERE 5
0.487733505222967
HERE 4
HERE 5
0.48572814491614125
HERE 4
HERE 5
0.4863761207153763
HERE 4
HERE 5
0.48434405467193375
HERE 4
HERE 5
0.48491475662156436
HERE 4
HERE 5
0.4828558041749158
HERE 4
HERE 5
0.4805075979170361
HERE 4
HERE 5
0.4810882405739889
HERE 4
HERE 5
0.4789886099070024
HERE 4
HERE

0.5762616157230287
HERE 4
HERE 5
0.599971650120354
HERE 4
HERE 5
0.5999417974709633
HERE 4
HERE 5
0.6055182750177508
HERE 4
HERE 5
0.6186891908193441
HERE 4
HERE 5
0.6211888263067014
HERE 4
HERE 5
0.6295726110256776
HERE 4
HERE 5
0.639686776515207
HERE 4
HERE 5
0.6476135301628245
HERE 4
HERE 5
0.6893350101116855
HERE 4
HERE 5
0.6994103252469692
HERE 4
HERE 5
0.6841469108323786
HERE 4
HERE 5
0.7038894133674605
HERE 4
HERE 5
0.6930834875172547
HERE 4
HERE 5
0.699911518018375
HERE 4
HERE 5
0.6804775112574344
HERE 4
HERE 5
0.6521262163791803
HERE 4
HERE 5
0.5738408295313508
HERE 4
HERE 5
0.5556816510834519
HERE 4
HERE 5
0.53912852952985
HERE 4
HERE 5
0.531186633587093
HERE 4
HERE 5
0.49381319121596157
HERE 4
HERE 5
0.4732112155977122
HERE 4
HERE 5
0.41972530795790597
HERE 4
HERE 5
0.38987295752901474
HERE 4
HERE 5
0.360890963476562
HERE 4
HERE 5
0.35760685863175024
HERE 4
HERE 5
0.23121119465901963
HERE 4
HERE 5
0.1720242599077224
HERE 4
HERE 5
0.15402494968804525
HERE 4
HERE 5
0.137976755